In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.utils import class_weight

After having tuned the parameters of the models on the improvements to the dataset the pickled model and a specialized model are used to develop new data sets containing the predictions and the scores associated with each actor.

In [2]:
df = pd.read_csv('best_actor_hd.csv')
general_model = pickle.load(open('finer_predictions_2.dat','rb'))
special_model = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=11, monotone_constraints='()',
              n_estimators=2000, n_jobs=0, num_class=3, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=0.9,
              tree_method='exact', validate_parameters=1, verbosity=None)


The general model buffer is loaded, as well as a base special model that will be trained on a set excluding the performances of each particular actor.

In [7]:
def balancedWeights(label_set):
    classes = np.unique(label_set)
    class_weights = class_weight.compute_class_weight('balanced',
                                                           classes,
                                                           label_set)
    
    cw_dict = {classes[i]: class_weights[i] for i in range(len(classes))}
    cw_array = [cw_dict[i] for i in label_set.values]
    return cw_array

def roc_auc_ovo_try(targets, probabilities):
    try:
        return metrics.roc_auc_score(targets, probabilities, average='weighted',multi_class='ovo',labels=[0,1,2])
    except ZeroDivisionError:
        return 0.0

In [8]:
feat_list = ['score_1','score_2','metascore','avg_vote','weighted_average_vote','mean_vote','median_vote',
        'males_allages_avg_vote','males_allages_votes','females_allages_avg_vote','females_allages_votes',
       'top1000_voters_rating','top1000_voters_votes','us_voters_rating','us_voters_votes','non_us_voters_rating',
       'non_us_voters_votes','age_at_performance','year','height','ordering','num_reviews','polarity_mean','objectivity_mean',
       'budget','worlwide_gross_income','duration','numeric_key']
actor_list = df.actor.unique().tolist()

In [9]:
predictions_df = pd.DataFrame()
score_list = []

An empty dataframe and an empty list will have all subsequent information appended to them to be later transformed into/saved as new data sets for visualizing the different predictions.

In [10]:
for person in actor_list:
    non_actor_set = df[df.actor != person].copy()
    actor_set = df[df.actor == person].copy()
    actor_view = actor_set[['actor','title','year','age_at_performance','award']].copy()
    actor_view.reset_index(drop=True, inplace=True)
    
    X_train_non, y_train_non = non_actor_set[feat_list], non_actor_set['award']
    X_test_actor, y_test_actor = actor_set[feat_list], actor_set['award']
    
    print(y_test_actor)
    non_actor_weights = balancedWeights(y_train_non)
    non_actor_weights = balancedWeights(y_train_non)
    
    special_model.fit(X_train_non, y_train_non, eval_set=[(X_train_non,y_train_non),(X_test_actor,y_test_actor)], sample_weight=non_actor_weights, early_stopping_rounds=100)
    general_predictions = general_model.predict(X_test_actor)
    general_probabilities = general_model.predict_proba(X_test_actor)
    special_predictions = special_model.predict(X_test_actor)
    special_probabilities = special_model.predict_proba(X_test_actor)
    
    preds_frame = pd.DataFrame({'gm_preds':general_predictions,'sm_preds':special_predictions})
    preds_frame['mean_pred'] = np.select([preds_frame.gm_preds == preds_frame.sm_preds, preds_frame.gm_preds != preds_frame.sm_preds],[preds_frame.gm_preds, actor_view.award], default=actor_view.award)
    comparison_frame = pd.concat([actor_view,preds_frame], axis=1)
    predictions_df = pd.concat([predictions_df,comparison_frame],axis=0)
    
    
    #format for this is [actor_name, gm_accuracy, gm_roc, sm_accuracy, sm_roc]
    score_row = [person,
                 '{:.2%}'.format(metrics.balanced_accuracy_score(y_test_actor,general_predictions)),
                '{:.4}'.format(roc_auc_ovo_try(y_test_actor,general_probabilities)),
                '{:.2%}'.format(metrics.balanced_accuracy_score(y_test_actor,special_predictions)),
                '{:.4}'.format(roc_auc_ovo_try(y_test_actor,special_probabilities))]
    score_list.append(score_row)
    

0       1.0
7128    0.0
7129    0.0
7130    0.0
7131    0.0
7132    0.0
7133    0.0
7134    0.0
7135    0.0
7136    0.0
7137    0.0
7138    0.0
7139    0.0
7140    0.0
Name: award, dtype: float64
[0]	validation_0-merror:0.25571	validation_1-merror:0.57143
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 100 rounds.
[1]	validation_0-merror:0.22808	validation_1-merror:0.57143
[2]	validation_0-merror:0.21583	validation_1-merror:0.50000
[3]	validation_0-merror:0.22364	validation_1-merror:0.57143


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0. 1. 2.], y=1       1.0
2       2.0
3       1.0
4       1.0
5       1.0
       ... 
8335    0.0
8336    0.0
8337    0.0
8338    0.0
8339    0.0
Name: award, Length: 8326, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[4]	validation_0-merror:0.21271	validation_1-merror:0.50000
[5]	validation_0-merror:0.21727	validation_1-merror:0.50000
[6]	validation_0-merror:0.21175	validation_1-merror:0.50000
[7]	validation_0-merror:0.21043	validation_1-merror:0.50000
[8]	validation_0-merror:0.21871	validation_1-merror:0.50000
[9]	validation_0-merror:0.21883	validation_1-merror:0.50000
[10]	validation_0-merror:0.21823	validation_1-merror:0.50000
[11]	validation_0-merror:0.21283	validation_1-merror:0.50000
[12]	validation_0-merror:0.21319	validation_1-merror:0.50000
[13]	validation_0-merror:0.21199	validation_1-merror:0.50000
[14]	validation_0-merror:0.21367	validation_1-merror:0.50000
[15]	validation_0-merror:0.21271	validation_1-merror:0.50000
[16]	validation_0-merror:0.20790	validation_1-merror:0.50000
[17]	validation_0-merror:0.20562	validation_1-merror:0.50000
[18]	validation_0-merror:0.20610	validation_1-merror:0.50000
[19]	validation_0-merror:0.20598	validation_1-merror:0.50000
[20]	validation_0-merror:0.207

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0. 1. 2.], y=0       1.0
2       2.0
3       1.0
4       1.0
5       1.0
       ... 
8335    0.0
8336    0.0
8337    0.0
8338    0.0
8339    0.0
Name: award, Length: 8287, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[3]	validation_0-merror:0.21274	validation_1-merror:0.20755
[4]	validation_0-merror:0.22409	validation_1-merror:0.20755
[5]	validation_0-merror:0.21902	validation_1-merror:0.18868
[6]	validation_0-merror:0.22324	validation_1-merror:0.20755
[7]	validation_0-merror:0.22409	validation_1-merror:0.16981
[8]	validation_0-merror:0.21600	validation_1-merror:0.18868
[9]	validation_0-merror:0.21781	validation_1-merror:0.18868
[10]	validation_0-merror:0.21516	validation_1-merror:0.18868
[11]	validation_0-merror:0.21250	validation_1-merror:0.18868
[12]	validation_0-merror:0.21190	validation_1-merror:0.15094
[13]	validation_0-merror:0.21105	validation_1-merror:0.16981
[14]	validation_0-merror:0.21069	validation_1-merror:0.15094
[15]	validation_0-merror:0.20574	validation_1-merror:0.15094
[16]	validation_0-merror:0.20816	validation_1-merror:0.16981
[17]	validation_0-merror:0.20526	validation_1-merror:0.16981
[18]	validation_0-merror:0.20176	validation_1-merror:0.15094
[19]	validation_0-merror:0.2027

...

[3]	validation_0-merror:0.23827	validation_1-merror:0.00000
[4]	validation_0-merror:0.23047	validation_1-merror:0.00000
[5]	validation_0-merror:0.23251	validation_1-merror:0.00000
[6]	validation_0-merror:0.22963	validation_1-merror:0.00000
[7]	validation_0-merror:0.22831	validation_1-merror:0.00000
[8]	validation_0-merror:0.22520	validation_1-merror:0.00000
[9]	validation_0-merror:0.22484	validation_1-merror:0.00000
[10]	validation_0-merror:0.22412	validation_1-merror:0.00000
[11]	validation_0-merror:0.22387	validation_1-merror:0.00000
[12]	validation_0-merror:0.22471	validation_1-merror:0.00000
[13]	validation_0-merror:0.22064	validation_1-merror:0.00000
[14]	validation_0-merror:0.22028	validation_1-merror:0.00000
[15]	validation_0-merror:0.21944	validation_1-merror:0.00000
[16]	validation_0-merror:0.21644	validation_1-merror:0.00000
[17]	validation_0-merror:0.21224	validation_1-merror:0.00000
[18]	validation_0-merror:0.21248	validation_1-merror:0.00000
[19]	validation_0-merror:0.2115

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0. 1. 2.], y=0       1.0
1       1.0
2       2.0
3       1.0
4       1.0
       ... 
8335    0.0
8336    0.0
8337    0.0
8338    0.0
8339    0.0
Name: award, Length: 8305, dtype: float64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


[2]	validation_0-merror:0.23745	validation_1-merror:0.17143
[3]	validation_0-merror:0.22565	validation_1-merror:0.17143
[4]	validation_0-merror:0.22757	validation_1-merror:0.17143
[5]	validation_0-merror:0.22649	validation_1-merror:0.17143
[6]	validation_0-merror:0.22143	validation_1-merror:0.17143
[7]	validation_0-merror:0.22192	validation_1-merror:0.17143
[8]	validation_0-merror:0.21589	validation_1-merror:0.17143
[9]	validation_0-merror:0.21276	validation_1-merror:0.17143
[10]	validation_0-merror:0.21168	validation_1-merror:0.14286
[11]	validation_0-merror:0.21361	validation_1-merror:0.14286
[12]	validation_0-merror:0.21096	validation_1-merror:0.17143
[13]	validation_0-merror:0.21264	validation_1-merror:0.17143
[14]	validation_0-merror:0.21072	validation_1-merror:0.17143
[15]	validation_0-merror:0.20650	validation_1-merror:0.14286
[16]	validation_0-merror:0.20397	validation_1-merror:0.14286
[17]	validation_0-merror:0.20578	validation_1-merror:0.14286
[18]	validation_0-merror:0.20747

[136]	validation_0-merror:0.16580	validation_1-merror:0.11429
[137]	validation_0-merror:0.16532	validation_1-merror:0.11429
[138]	validation_0-merror:0.16472	validation_1-merror:0.08571
[139]	validation_0-merror:0.16400	validation_1-merror:0.08571
[140]	validation_0-merror:0.16376	validation_1-merror:0.08571
[141]	validation_0-merror:0.16352	validation_1-merror:0.08571
[142]	validation_0-merror:0.16364	validation_1-merror:0.08571
[143]	validation_0-merror:0.16340	validation_1-merror:0.08571
[144]	validation_0-merror:0.16303	validation_1-merror:0.08571
[145]	validation_0-merror:0.16303	validation_1-merror:0.08571
[146]	validation_0-merror:0.16279	validation_1-merror:0.08571
[147]	validation_0-merror:0.16279	validation_1-merror:0.08571
[148]	validation_0-merror:0.16243	validation_1-merror:0.08571
[149]	validation_0-merror:0.16279	validation_1-merror:0.08571
[150]	validation_0-merror:0.16243	validation_1-merror:0.08571
[151]	validation_0-merror:0.16243	validation_1-merror:0.08571
[152]	va

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [11]:
score_list

[['Adam Driver', '96.15%', '1.0', '26.92%', '0.7692'],
 ['Adolphe Menjou', '100.00%', '1.0', '97.12%', '0.9904'],
 ['Adrien Brody', '100.00%', '1.0', '46.88%', '0.9688'],
 ['Al Pacino', '64.91%', '0.9761', '82.02%', '0.9291'],
 ['Alan Arkin', '98.84%', '1.0', '97.67%', '1.0'],
 ['Alan Bates', '98.39%', '1.0', '38.71%', '0.6129'],
 ['Albert Finney', '98.28%', '1.0', '78.88%', '0.8836'],
 ['Alec Guinness', '97.85%', '0.9844', '86.02%', '0.9062'],
 ['Alexander Knox', '100.00%', '0.0', '0.00%', '0.0'],
 ['Alfred Lunt', '100.00%', '1.0', '50.00%', '0.0'],
 ['Andrew Garfield', '95.00%', '1.0', '25.00%', '0.6'],
 ['Anthony Franciosa', '96.15%', '1.0', '46.15%', '0.8462'],
 ['Anthony Hopkins', '97.27%', '0.9799', '95.08%', '0.938'],
 ['Anthony Quinn', '100.00%', '1.0', '43.83%', '0.8056'],
 ['Antonio Banderas', '99.07%', '1.0', '99.07%', '1.0'],
 ['Art Carney', '100.00%', '1.0', '46.15%', '1.0'],
 ['Arthur Kennedy', '100.00%', '1.0', '48.65%', '0.8649'],
 ['Barry Fitzgerald', '95.00%', '0.975'

In [12]:
predictions_df

,actor,title,year,age_at_performance,award,gm_preds,sm_preds,mean_pred
0,Adam Driver,marriage story,2019,36.0,1.0,1.0,2.0,1.0
1,Adam Driver,the dead dont die,2019,36.0,0.0,0.0,0.0,0.0
2,Adam Driver,the report,2019,36.0,0.0,0.0,1.0,0.0
3,Adam Driver,the man who killed don quixote,2018,35.0,0.0,0.0,0.0,0.0
4,Adam Driver,blackkklansman,2018,35.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...
30,Michael Keaton,touch and go,1986,35.0,0.0,0.0,0.0,0.0
31,Michael Keaton,gung ho,1986,35.0,0.0,0.0,0.0,0.0
32,Michael Keaton,johnny dangerously,1984,33.0,0.0,0.0,0.0,0.0
33,Michael Keaton,mr mom,1983,32.0,0.0,0.0,0.0,0.0


In [13]:
scores_df = pd.DataFrame(data=score_list, columns = ['actor','gm_accuracy','gm_roc','sm_accuracy','sm_roc'])

In [14]:
scores_df

,actor,gm_accuracy,gm_roc,sm_accuracy,sm_roc
0,Adam Driver,96.15%,1.0,26.92%,0.7692
1,Adolphe Menjou,100.00%,1.0,97.12%,0.9904
2,Adrien Brody,100.00%,1.0,46.88%,0.9688
3,Al Pacino,64.91%,0.9761,82.02%,0.9291
4,Alan Arkin,98.84%,1.0,97.67%,1.0
...,...,...,...,...,...
227,Lewis Stone,100.00%,0.0,98.46%,0.0
228,Oskar Werner,71.43%,0.0,42.86%,0.0
229,Clifton Webb,100.00%,0.0,75.00%,0.0
230,Massimo Troisi,100.00%,0.0,100.00%,0.0


In [15]:
scores_df.to_csv('prediction_metrics.csv')
predictions_df.to_csv('performance_predictions.csv')